## 0.概要

EfficientNetが以下のような3種類の画像を分類できるように訓練します。

![](./img/train_2022-05-29-17-06-08.png)

モデルの訓練には主に以下のものが必要です。  
  

・画像の前処理をするもの  
・Dataset(データをまとめるクラス)  
・DataLoader(データをモデルに入力するもの)  
・モデル  
・損失関数  
・オプティマイザー(モデルのパラメータの調整方法)

## 1.ライブラリのインポート

In [2]:
import glob
from PIL import Image
from tqdm import tqdm
import os
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
import json
from sklearn.model_selection import StratifiedKFold
#from efficientnet_pytorch import EfficientNet

import tensorboardX as tbx

## 2.乱数のシード（再現性の担保）

In [9]:
#seed 数字はなんでもいい
torch.manual_seed(999)
random.seed(999)
np.random.seed(999)
seed = 999

#Disabling the benchmark by CUDA convolution operation(GPUを使うときの再現性の担保) (https://pytorch.org/docs/stable/notes/randomness.html)
torch.backends.cudnn.benchmark = False 
torch.backends.cudnn.determinstic = True

動作確認

In [12]:
#0~1の乱数を出力
torch.rand(1)

tensor([0.0457])

## 3.辞書argsのロード

In [3]:
#ここにpathなどを記述
json_file = open('args.json','r')
args = json.load(json_file)

動作確認

In [14]:
args["model_name"]

'efficientnet-b4'

※データセットのRBG値の平均(mean),標準偏差(std)の求め方は、『PyTorch実践入門』の208ページなどをご覧ください。

![画像が表示されていないきは、vscodeの「ファイル」⇒「フォルダを開く」でこのファイルが入っているフォルダを開いてみてください](./img/train_2022-02-10-14-55-27.png)

## 4.デバイスを呼び出す関数

In [15]:
def check_GPU():
    """
    使用するデバイスを出力する関数

    Returns
    ----------
    device : object
        GPUが使えるなら'cuda:0',使えないなら'cpu'を返す
    """
    print("Check GPU")
    print(torch.cuda.is_available())
    print("-----")
    if torch.cuda.is_available():
        d_type = "cuda:0"
    else:
        d_type = "cpu"
    device = torch.device(d_type)
    return device 

動作確認

In [16]:
test = check_GPU()
test

Check GPU
False
-----


device(type='cpu')

## 5.画像の前処理クラス

In [41]:
class ImageTransform():
    """
    画像の前処理クラス。
    リサイズ → テンソル化 → 標準化
    訓練時だけ、データオーギュメンテーション(DA)ができるように、train,validで分けて書いた

    Attributes
    ----------
    resize : int
        画像のリサイズの値
    mean : (R, G, B)
        データセットのRGB値の各平均値
    std : (R, G, B)
        データセットのRGB値の各標準偏差
    """
    # 初期化メソッド(特殊メソッド)
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
            transforms.Resize((resize,resize)),  
            transforms.ToTensor(),  
            transforms.Normalize(mean, std)  
            ]),
            'valid': transforms.Compose([
            transforms.Resize((resize,resize)), 
            transforms.ToTensor(),  
            transforms.Normalize(mean, std)  
            ]),
        }

    #引数なしで呼ばれたときの挙動を定義(特殊メソッド)
    def __call__(self, img, phase):
        return self.data_transform[phase](img)

## 6.Datasets, DataLoader

In [49]:
class MeatDataset(data.Dataset):
    """
    画像のデータセットクラス。
    Pytorch Dataset class を継承

    Attributes
    ----------
    file_list : list
        画像のpath_list
    label_list : list
        ラベルのリスト
    transform : object
        class ImageTransform()
    phase : str
        'train' or 'valid'
    """

    def __init__(self, file_list, label_list, transform, phase):
        self.file_list = file_list
        self.label_list = label_list
        self.transform = transform  
        self.phase = phase  

    #このクラスの画像枚数を定義。
    def __len__(self):
        return len(self.file_list) 

    #このクラスに角括弧でアクセスした時の挙動を定義
    def __getitem__(self, index):
        # 画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  
        # 前処理
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        #label
        label = self.label_list[index]
        #Tensorに変換
        label = torch.tensor(label)         

        return img_transformed , label

In [19]:
def image_path_list(image_root_path, mode): 
    """
    画像をクラスごとに集める関数

    Parameters
    ----------
    image_root_path : path
        画像のroot_path. 
    mode : str
        'class0' or 'class1' or 'class2

    Returns
    ----------
    path_list : list
        画像のpath_list
    label_list : list
        0 or 1 or 2
    """

    path_list = glob.glob(os.path.join(image_root_path, mode ,'*.tif')) #画像path

    if mode=='class0':
        label = 0
    elif mode=='class1':
        label = 1
    elif mode=='class2':
        label = 2

    label_list = [label for _ in range(len(path_list))]
    """これと同じ意味
    label_list = []
    for _ in range(len(path_list)):
        label_list.append(label)"""

    return path_list, label_list

## 7.NetWork

In [15]:
def efficientnet(model_name):
    """
    efficientNetを持ってくる関数
    
    Parameters
    ----------
    model_name : str
        efficientnet-b4
    
    Returns
    ----------
    model : object
        訓練済efficientnetを出力
    """
    model = EfficientNet.from_pretrained(model_name)
    #modelの最終層の出力を3に変更する。
    model._fc  = nn.Linear(in_features=1792, out_features=3, bias=True)
    return model

動作確認

In [16]:
input = torch.randn((8,3,380,380))
input.size()

torch.Size([8, 3, 380, 380])

In [17]:
model = efficientnet(args['model_name'])
output = model(input)
output

Loaded pretrained weights for efficientnet-b4


tensor([[ 0.0083,  0.0367, -0.0092],
        [ 0.0588, -0.1134, -0.0585],
        [ 0.1589,  0.1101, -0.0817],
        [ 0.0284, -0.0097,  0.0696],
        [-0.0711,  0.0835,  0.1356],
        [-0.0330,  0.0514,  0.0783],
        [ 0.0547, -0.2173,  0.1230],
        [ 0.1484, -0.0296,  0.0045]], grad_fn=<AddmmBackward>)

## 8.損失関数

In [33]:
#クロスエントロピー誤差関数
criterion = nn.CrossEntropyLoss()

![画像が表示されていないきは、vscodeの「ファイル」⇒「フォルダを開く」でこのファイルが入っているフォルダを開いてみてください](./img/train_2022-02-09-15-50-48.png)

動作確認

In [36]:
y = torch.tensor([0.1,0.8,0.1]).unsqueeze(0)
t = torch.tensor(2).unsqueeze(0)
print(y)
print(t)

tensor([[0.1000, 0.8000, 0.1000]])
tensor([2])


In [37]:
criterion(y,t)

tensor(1.3897)

## 9.オプティマイザー

In [ ]:
model = efficientnet(args['model_name'])

#optimizer
opt = torch.optim.Adam(model.parameters()) # model.parameters():モデルのパラメータ

玉の転がし方を決める役割のイメージ？

![画像が表示されていないきは、vscodeの「ファイル」⇒「フォルダを開く」でこのファイルが入っているフォルダを開いてみてください](./img/train_2022-02-10-08-42-51.png)

## 10.訓練

In [21]:
def train(model, dataloaders_dict, device, opt, criterion, writer, fold, save_root_path, me, le ):
    
    #ネットワークがある程度固定であれば、高速化
    torch.backends.cudnn.benchmark = True

    print("\nStarting training")
    for epoch in range(0, me+1):
        print('Epoch {}/{} fold{}'.format(epoch, me, fold))
        print('-------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに
            else:
                model.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和                
            correct = 0 #正解数
            total = 0 #データ数

            #未訓練時の性能評価
            if (epoch == 0 and phase=='train'):
                continue
            
            # データローダーからミニバッチを取り出すループ。args['batch_size']枚ごと取り出す
            for inputs,labels in tqdm(dataloaders_dict[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化。
                opt.zero_grad()

                # 順伝搬（forward）計算。
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算。
                    prediction = torch.argmax(outputs, dim=1)

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward() 
                        opt.step()

                # 損失の計算
                if phase == 'train':
                    epoch_loss += loss.item()*inputs.size(0)/4
                elif phase == 'valid':
                    epoch_loss += loss.item()*inputs.size(0)

                total += labels.shape[0] #データ数
                correct += int((prediction == labels).sum()) #正解数

            #accuracy
            accuracy = correct/total

            #tensorboardに出力
            if phase == 'train':
                writer.add_scalar(f'train_epoch_loss_fold{fold}', epoch_loss, epoch) #(グラフ名, y座標, x座標)
                writer.add_scalar(f'train_epoch_accuracy_fold{fold}', accuracy, epoch)                
            elif phase == 'valid':
                writer.add_scalar(f'valid_epoch_loss_fold{fold}', epoch_loss, epoch)
                writer.add_scalar(f'valid_epoch_accuracy_fold{fold}', accuracy, epoch)                  

            # epochごとのloss,accuracyを表示
            if epoch % le == 0:
                print(f'phase:{phase}')
                print("epoch = %4d   loss = %0.4f  accuracy = %0.4f" %(epoch, epoch_loss, accuracy))

        #epochごとに重みを保存
        if epoch % 1 ==0:
            save_path = os.path.join(save_root_path, 'model_weight', f'fold{fold}_epoch={epoch}.pth')
            torch.save(model.state_dict(), save_path)

## 11.クロスバリデーション

![画像が表示されていないきは、vscodeの「ファイル」⇒「フォルダを開く」でこのファイルが入っているフォルダを開いてみてください](./img/train_2022-02-09-16-39-13.png)

In [38]:
# cross validation 各Foldごとでラベルは均等にわけられる
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

## 12.main

In [31]:
def main(args = args):
    #GPU
    device = check_GPU()

    #画像の前処理クラス
    transform = ImageTransform(args['resize'], args['mean'], args['std'])
        
    os.makedirs(os.path.join(args['save_root_path'], 'model_weight'), exist_ok=True) #重みファイルの保存

    #画像pathを集める
    path0,label0 = image_path_list(args['image_root_path'], mode='class0')
    path1, label1 = image_path_list(args['image_root_path'], mode='class1')
    path2, label2 = image_path_list(args['image_root_path'], mode='class2')

    #すべてのパス,ラベルをまとめる
    path = path0 + path1 + path2
    label = label0 + label1 + label2

    #criterion
    criterion = nn.CrossEntropyLoss()
        
    # cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    #logger
    writer = tbx.SummaryWriter()

    for fold , (train_idx, valid_idx) in enumerate(skf.split(path,label)): #loop Fold
        train_path = [path[i] for i in train_idx]
        valid_path = [path[i] for i in valid_idx]
        train_label = [label[i] for i in train_idx] 
        valid_label = [label[i] for i in valid_idx]

        #Dataset
        train_dataset  = MeatDataset(train_path,train_label,transform,phase='train') 
        valid_dataset = MeatDataset(valid_path,valid_label,transform,phase='valid')

        #DataLoder
        train_dataloader = torch.utils.data.DataLoader(
        train_dataset,batch_size=args['batch_size'],shuffle=True
        )
        valid_dataloader = torch.utils.data.DataLoader(
        valid_dataset,batch_size=1,shuffle=False
        )                  

        # 辞書型変数にまとめる
        dataloaders_dict = {"train": train_dataloader, "valid": valid_dataloader}        

        #Network
        model = efficientnet(args['model_name'])
        model.to(device)

        #optimizer
        opt = torch.optim.Adam(model.parameters())

        #train
        train(model, dataloaders_dict, device, opt, criterion, writer, fold, args['save_root_path'], args['max_epoch'], args['log_every'])

    writer.close()

※ TensorBoardで学習結果を確認するには、コマンド  
tensorboard --logdir ’runsフォルダのパス’  
と入力して、http://localhost:6006/ にアクセス

In [ ]:
main()